## Hücre 1

# Generate DS2 Complete TMN (Google Colab)

This notebook only generates the DS2 Complete TMN outputs directly on Google Drive. Code is cloned from GitHub into /content/omr_tmn, dataset lives on Google Drive at SRC_ROOT. We run shard-wise TMN placement and verify counts. There is no training, no CUDA usage, and no Mask R-CNN code.


In [ ]:
# Hücre 2

# Connect to Google Drive
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    # Use force_remount to avoid stale mounts/timeouts
    drive.mount('/content/drive', force_remount=True)
    print('Drive mounted at /content/drive')
else:
    print('Not in Colab; skipping Drive mount.')

In [ ]:
# Hücre 3

# Clone repo from GitHub (optional if already in Drive)
import os, sys, subprocess, pathlib

REPO_URL = 'https://github.com/mtalhabalci/omr_tmn.git'
WORKDIR = '/content/omr_tmn'
SCRIPT_PATH = None

if 'google.colab' in sys.modules:
    if not os.path.isdir(WORKDIR):
        print('Cloning repo...')
        subprocess.check_call(['git','clone',REPO_URL, WORKDIR])
    else:
        print('Repo exists; pulling latest...')
        subprocess.call(['bash','-lc', f'cd {WORKDIR} && git pull --rebase'])
    # Change directory via Python for reliability
    os.chdir(WORKDIR)
    print('Changed dir to', os.getcwd())
    # Prefer src/place_tmn_batch.py inside repo
    candidate = os.path.join(WORKDIR, 'src', 'place_tmn_batch.py')
    if os.path.isfile(candidate):
        SCRIPT_PATH = candidate
    else:
        alt = os.path.join(WORKDIR, 'place_tmn_batch.py')
        SCRIPT_PATH = alt if os.path.isfile(alt) else candidate
    print({'SCRIPT_PATH': SCRIPT_PATH, 'exists': os.path.isfile(SCRIPT_PATH)})
else:
    print('Not in Colab; skipping clone and cd.')

In [ ]:
# Hücre 4

# Configure Drive paths for DS2 Complete (source) and DS2 Complete TMN (output)
SRC_ROOT = '/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete'
OUT_ROOT = '/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn'
SRC_IMAGES = f"{SRC_ROOT}/images"
print({'SRC_ROOT': SRC_ROOT, 'OUT_ROOT': OUT_ROOT, 'SRC_IMAGES': SRC_IMAGES})

In [ ]:
# Hücre 5

# Run TMN placement on DS2 Complete: shard-wise, resume-safe, JSONs at OUT_ROOT root
import sys, subprocess, os, glob, time

# Resolve script path robustly
sp_candidates = []
try:
    # If set in previous cell
    if 'SCRIPT_PATH' in globals() and isinstance(SCRIPT_PATH, str):
        sp_candidates.append(SCRIPT_PATH)
except Exception:
    pass
sp_candidates += [
    os.path.join('/content/omr_tmn', 'src', 'place_tmn_batch.py'),
    os.path.join('/content/omr_tmn', 'place_tmn_batch.py'),
    'src/place_tmn_batch.py'
]
SCRIPT_TO_RUN = next((p for p in sp_candidates if os.path.isfile(p)), None)
print({'SCRIPT_TO_RUN': SCRIPT_TO_RUN, 'exists': bool(SCRIPT_TO_RUN and os.path.isfile(SCRIPT_TO_RUN))})
if not SCRIPT_TO_RUN:
    print('ERROR: place_tmn_batch.py not found in expected locations. Check repo structure.')

# Ensure output structure exists (mirror source)
os.makedirs(os.path.join(OUT_ROOT, 'images'), exist_ok=True)
os.makedirs(os.path.join(OUT_ROOT, 'segmentation'), exist_ok=True)
os.makedirs(os.path.join(OUT_ROOT, 'instance'), exist_ok=True)
LOG_DIR = os.path.join(OUT_ROOT, 'logs')
os.makedirs(LOG_DIR, exist_ok=True)

# Collect JSON shards at SRC_ROOT root (not under jsonlar)
train_shards = sorted(glob.glob(os.path.join(SRC_ROOT, 'deepscores-complete-*_train.json')))
test_shards = sorted(glob.glob(os.path.join(SRC_ROOT, 'deepscores-complete-*_test.json')))
shards = train_shards + test_shards
print({'shard_count': len(shards), 'sample_shards': [os.path.basename(x) for x in shards[:5]]})

# Process each shard; skip if .done exists; within shard, reprocess regardless of existing outputs (force)
for jp in shards:
    base = os.path.basename(jp)
    done_marker = os.path.join(LOG_DIR, base + '.done')
    if os.path.exists(done_marker):
        print('Skipping shard, already done:', base)
        continue
    t0 = time.time()
    if SCRIPT_TO_RUN:
        cmd = [
            sys.executable, '-u', SCRIPT_TO_RUN,
            '--images-dir', SRC_IMAGES,
            '--out-root', OUT_ROOT,
            '--json-path', jp,
            '--checkpoint', '200',
            '--json-out-mode', 'per-shard',
            '--symbols-dir', '/content/omr_tmn/tmn_symbols_png',
            '--force',
            '--slot-w', '12',
            '--slot-h', '36',
            '--disable-barline-mask',
            '--limit', '50'
        ]
    else:
        cmd = [sys.executable, '-u', '-c', "import sys; print('script missing'); sys.exit(2)"]

    print('Running shard:', base)
    print('Command:', ' '.join(cmd))

    # Live log streaming + timeout
    TIMEOUT_PER_SHARD = int(os.environ.get('TMN_TIMEOUT_SEC', '3600'))  # 1 hour default
    HEARTBEAT_SEC = int(os.environ.get('TMN_HEARTBEAT_SEC', '30'))
    last_beat = time.time()
    combined_tail = []

    ret = None
    try:
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
        while True:
            line = p.stdout.readline()
            if not line:
                if p.poll() is not None:
                    break
                # idle short sleep to avoid tight loop
                time.sleep(0.2)
                continue
            line = line.rstrip('\n')
            if line:
                print(line)
                combined_tail.append(line)
                if len(combined_tail) > 400:
                    combined_tail = combined_tail[-400:]

            now = time.time()
            if now - last_beat >= HEARTBEAT_SEC:
                elapsed = int(now - t0)
                print(f"[heartbeat] {base} running {elapsed}s...")
                last_beat = now

            if now - t0 > TIMEOUT_PER_SHARD:
                print(f"Timeout after {TIMEOUT_PER_SHARD}s; terminating shard {base}")
                try:
                    p.terminate()
                    p.wait(timeout=10)
                except Exception:
                    pass
                try:
                    p.kill()
                except Exception:
                    pass
                ret = 124  # conventional timeout code
                break

        if ret is None:
            ret = p.wait()
    except Exception as e:
        print('Runner failed:', e)
        ret = 2

    # Always show a brief tail snippet for summary/diagnostics
    tail_snip = combined_tail[-20:]
    if tail_snip:
        print('--- tail (last 20 lines) ---')
        for ln in tail_snip:
            print(ln)
        print('--- end tail ---')

    # If the placer wrote JSONs under OUT_ROOT/jsonlar, move them to OUT_ROOT root
    moved = []
    jsonlar_dir = os.path.join(OUT_ROOT, 'jsonlar')
    if os.path.isdir(jsonlar_dir):
        for f in glob.glob(os.path.join(jsonlar_dir, '*.json')):
            dest = os.path.join(OUT_ROOT, os.path.basename(f))
            try:
                os.replace(f, dest)
                moved.append(os.path.basename(f))
            except Exception as e:
                print('Move failed', f, '->', dest, e)
        try:
            if not os.listdir(jsonlar_dir):
                os.rmdir(jsonlar_dir)
        except Exception:
            pass

    # Quick count of newly created outputs since shard start
    def count_new(dir_path, suffix):
        cnt = 0
        try:
            for nm in os.listdir(dir_path):
                if not nm.lower().endswith(suffix):
                    continue
                pth = os.path.join(dir_path, nm)
                try:
                    st = os.stat(pth)
                    if st.st_mtime >= t0:
                        cnt += 1
                except Exception:
                    pass
        except Exception:
            pass
        return cnt
    new_imgs = count_new(os.path.join(OUT_ROOT, 'images'), '.png')
    new_segs = count_new(os.path.join(OUT_ROOT, 'segmentation'), '_seg.png')
    new_insts = count_new(os.path.join(OUT_ROOT, 'instance'), '_inst.png')

    dur = round(time.time() - t0, 1)
    try:
        with open(os.path.join(LOG_DIR, 'run.csv'), 'a', encoding='utf-8') as lf:
            lf.write(f"{base},{ret},{dur},{len(moved)}\n")
    except Exception as e:
        print('Log write failed:', e)

    if ret == 0:
        try:
            with open(done_marker, 'w', encoding='utf-8') as dm:
                dm.write('ok')
        except Exception as e:
            print('Done marker write failed:', e)

    print({'shard': base, 'exit': ret, 'sec': dur, 'moved_jsons': moved[:3], 'new_outputs': {'images': new_imgs, 'seg': new_segs, 'inst': new_insts}})

In [ ]:
# Hücre 6

# Verify outputs for DS2 Complete TMN
import os, glob, json

img_count = len(glob.glob(f"{OUT_ROOT}/images/*.png"))
seg_count = len(glob.glob(f"{OUT_ROOT}/segmentation/*_seg.png"))
inst_count = len(glob.glob(f"{OUT_ROOT}/instance/*_inst.png"))
# JSONs are expected at OUT_ROOT root (no jsonlar folder)
json_files = sorted(glob.glob(f"{OUT_ROOT}/*_train.json") + glob.glob(f"{OUT_ROOT}/*_test.json"))
print({'images': img_count, 'segmentation': seg_count, 'instance': inst_count, 'jsons_sample': [os.path.basename(x) for x in json_files[:5]], 'json_total': len(json_files)})

# Optional: consistency by basename
basename_noext = lambda p: os.path.splitext(os.path.basename(p))[0]
base_images = {basename_noext(p) for p in glob.glob(f"{OUT_ROOT}/images/*.png")}
base_segs = {os.path.basename(p).replace('_seg.png','') for p in glob.glob(f"{OUT_ROOT}/segmentation/*_seg.png")}
base_insts = {os.path.basename(p).replace('_inst.png','') for p in glob.glob(f"{OUT_ROOT}/instance/*_inst.png")}

if base_images:
    missing_seg = sorted(list(base_images - base_segs))
    missing_inst = sorted(list(base_images - base_insts))
    print({'images_vs_seg_equal': len(missing_seg) == 0, 'images_vs_inst_equal': len(missing_inst) == 0})
    if missing_seg[:5]: print('Missing segmentation (first 5):', missing_seg[:5])
    if missing_inst[:5]: print('Missing instance (first 5):', missing_inst[:5])

In [ ]:
# Hücre 7

# Inspect logs and shard detection to diagnose quick finish
import os, glob, csv
print({'SRC_ROOT': SRC_ROOT, 'OUT_ROOT': OUT_ROOT})

# Detect shards with primary and fallback patterns
train_shards = sorted(glob.glob(os.path.join(SRC_ROOT, 'deepscores-complete-*_train.json')))
test_shards = sorted(glob.glob(os.path.join(SRC_ROOT, 'deepscores-complete-*_test.json')))
if not train_shards and not test_shards:
    train_shards = sorted(glob.glob(os.path.join(SRC_ROOT, '*train*.json')))
    test_shards = sorted(glob.glob(os.path.join(SRC_ROOT, '*test*.json')))
shards = train_shards + test_shards
print({'shard_count': len(shards), 'sample_shards': [os.path.basename(x) for x in shards[:5]]})

# Logs summary
LOG_DIR = os.path.join(OUT_ROOT, 'logs')
run_csv = os.path.join(LOG_DIR, 'run.csv')
done_markers = sorted(glob.glob(os.path.join(LOG_DIR, '*.done')))
print({'log_dir_exists': os.path.isdir(LOG_DIR), 'done_markers': len(done_markers), 'done_samples': [os.path.basename(x) for x in done_markers[:5]]})

tail = []
stats = {'total_lines': 0, 'ret0': 0, 'ret_nonzero': 0}
if os.path.isfile(run_csv):
    with open(run_csv, 'r', encoding='utf-8') as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    stats['total_lines'] = len(lines)
    tail = lines[-10:]
    for ln in lines:
        # expected format: base,ret,dur,movedCount
        parts = ln.split(',')
        if len(parts) >= 4:
            try:
                ret = int(parts[1])
                stats['ret0'] += 1 if ret == 0 else 0
                stats['ret_nonzero'] += 1 if ret != 0 else 0
            except:
                pass
print({'run_csv_exists': os.path.isfile(run_csv), 'stats': stats})
print('run.csv last 10 lines:')
for ln in tail:
    print('  ', ln)

# Quick output presence check
img_count = len(glob.glob(f"{OUT_ROOT}/images/*.png"))
seg_count = len(glob.glob(f"{OUT_ROOT}/segmentation/*_seg.png"))
inst_count = len(glob.glob(f"{OUT_ROOT}/instance/*_inst.png"))
json_files = sorted(glob.glob(f"{OUT_ROOT}/*_train.json") + glob.glob(f"{OUT_ROOT}/*_test.json"))
print({'images': img_count, 'segmentation': seg_count, 'instance': inst_count, 'json_total': len(json_files), 'json_samples': [os.path.basename(x) for x in json_files[:5]]})

# Common causes summary hints
if len(shards) == 0:
    print('Hint: No shards detected at SRC_ROOT. Check JSON naming/patterns and SRC_ROOT path.')
elif len(done_markers) == len(shards):
    print('Hint: All shards show as done; the runner skipped them. Delete .done markers to re-run a shard if needed.')
elif stats['ret0'] == 0 and stats['total_lines'] > 0:
    print('Hint: Non-zero return codes; check repo availability or src/place_tmn_batch.py path.')
else:
    print('Hint: If durations are very small and moved_jsons=0, there might be nothing new to process (already placed).')

In [ ]:
# Hücre 8

# TMN sembolleri ve ilerleme loglarını kontrol et
import os, glob, json, time

SYM_DIR = '/content/omr_tmn/tmn_symbols_png'
print({'symbols_dir': SYM_DIR, 'exists': os.path.isdir(SYM_DIR)})
if os.path.isdir(SYM_DIR):
    try:
        names = [n for n in os.listdir(SYM_DIR) if n.lower().endswith('.png')]
        print({'symbol_png_count': len(names), 'samples': sorted(names)[:8]})
    except Exception as e:
        print('Symbols listing failed:', e)
else:
    print('Warning: TMN sembol klasörü yok; yerleştirme olmaz. Klasörü kontrol edin.')

# Çıktı klasörlerinde son bir saat içinde üretilen dosya sayısı
OUT_IMAGES = os.path.join(OUT_ROOT, 'images')
OUT_SEG = os.path.join(OUT_ROOT, 'segmentation')
OUT_INST = os.path.join(OUT_ROOT, 'instance')
cutoff = time.time() - 3600  # son 1 saat

def count_recent(dir_path, suffix=None):
    cnt = 0
    try:
        for nm in os.listdir(dir_path):
            if suffix and not nm.lower().endswith(suffix):
                continue
            p = os.path.join(dir_path, nm)
            try:
                if os.stat(p).st_mtime >= cutoff:
                    cnt += 1
            except Exception:
                pass
    except Exception:
        pass
    return cnt

print({'recent_outputs_last_1h': {
    'images': count_recent(OUT_IMAGES, '.png'),
    'segmentation': count_recent(OUT_SEG, '_seg.png'),
    'instance': count_recent(OUT_INST, '_inst.png')
}})

# Progress log (jsonlar/progress.jsonl) son 10 kayıt
PROG = os.path.join(OUT_ROOT, 'jsonlar', 'progress.jsonl')
print({'progress_log_exists': os.path.isfile(PROG), 'path': PROG})
if os.path.isfile(PROG):
    try:
        with open(PROG, 'r', encoding='utf-8') as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        tail = lines[-10:]
        print('progress.jsonl last 10:')
        for ln in tail:
            print('  ', ln)
    except Exception as e:
        print('Progress read failed:', e)

# Ortam değişkenleri (timeout/heartbeat)
import os
print({'TMN_TIMEOUT_SEC': os.environ.get('TMN_TIMEOUT_SEC', '3600'), 'TMN_HEARTBEAT_SEC': os.environ.get('TMN_HEARTBEAT_SEC', '30')})

In [ ]:
# Hücre 9

# DS2 shard JSON içeriğini doğrula: kategori isimleri, örnek annotation'lar ve filename -> dosya var mı
import os, json, glob

sample_jsons = sorted(glob.glob(os.path.join(SRC_ROOT, 'deepscores-complete-*_train.json')) + glob.glob(os.path.join(SRC_ROOT, 'deepscores-complete-*_test.json')))
print({'json_count': len(sample_jsons), 'first': os.path.basename(sample_jsons[0]) if sample_jsons else None})

if sample_jsons:
    jp = sample_jsons[0]
    with open(jp, 'r', encoding='utf-8') as f:
        data = json.load(f)
    cats = data.get('categories') or {}
    cat_names = []
    for k, v in cats.items():
        nm = v.get('name') if isinstance(v, dict) else None
        if isinstance(nm, str):
            cat_names.append(nm.lower())
    cat_names = sorted(set(cat_names))
    print({'unique_category_names_count': len(cat_names), 'has_notehead': any('notehead' in n for n in cat_names), 'has_rest': any('rest' in n for n in cat_names), 'has_flag': any('flag' in n for n in cat_names), 'has_accidental': any('accidental' in n for n in cat_names)})

    imgs = data.get('images') or []
    if isinstance(imgs, dict):
        imgs = list(imgs.values())
    samples = imgs[:10]
    exists = []
    for im in samples:
        fn = im.get('filename') or im.get('file_name')
        p = os.path.join(SRC_IMAGES, fn) if fn else None
        exists.append({'filename': fn, 'exists': os.path.isfile(p)})
    print({'image_samples_exists': exists})

    anns = data.get('annotations') or []
    if isinstance(anns, dict):
        anns = list(anns.values())
    # İlk 50 annotation içinden notehead içerenleri say
    def ann_names(a):
        cats = a.get('cat_id') or a.get('category_ids') or []
        ids = []
        if isinstance(cats, (list, tuple)):
            for c in cats:
                try:
                    ids.append(int(c))
                except Exception:
                    pass
        elif isinstance(cats, str):
            try:
                ids.append(int(cats))
            except Exception:
                pass
        # id->name map
        id2name = {}
        for k, v in (data.get('categories') or {}).items():
            try:
                cid = int(k)
            except Exception:
                continue
            nm = v.get('name') if isinstance(v, dict) else None
            if isinstance(nm, str):
                id2name[cid] = nm.lower()
        return [id2name.get(ci, str(ci)) for ci in ids]

    notehead_anns = 0
    for a in anns[:2000]:  # ilk 2000'de arayalım
        names = ann_names(a)
        if any('notehead' in n for n in names):
            notehead_anns += 1
    print({'notehead_annotations_in_first_2000': notehead_anns})
else:
    print('Uyarı: SRC_ROOT altında shard JSON bulunamadı.')

In [ ]:
# Hücre 10

# place_tmn_batch.py içine barline mask devre dışı bırakma bayrağı ekle
import os
SCRIPT_FILE = '/content/omr_tmn/src/place_tmn_batch.py'
print({'script_path': SCRIPT_FILE, 'exists': os.path.isfile(SCRIPT_FILE)})
if not os.path.isfile(SCRIPT_FILE):
    print('Uyarı: Script bulunamadı; Hücre 3 ve script yolu çıktısını kontrol edin.')
else:
    with open(SCRIPT_FILE, 'r', encoding='utf-8') as f:
        src = f.read()
    changed = False
    # Global değişken ekle
    if 'BARLINE_MASK_DISABLED' not in src:
        src = src.replace('from PIL import Image, ImageDraw,\nImageFilter',
                          'from PIL import Image, ImageDraw,\nImageFilter\n\nBARLINE_MASK_DISABLED = False')
        changed = True
    # Argparse bayrağı ekle
    if '--disable-barline-mask' not in src:
        src = src.replace(
            "ap.add_argument('--json-out-mode', type=str, default='single', choices=['single','per-shard'],\nhelp='Write a single merged JSON or one per input shard (when using\njson-glob)')",
            "ap.add_argument('--json-out-mode', type=str, default='single', choices=['single','per-shard'],\nhelp='Write a single merged JSON or one per input shard (when using\njson-glob)')\n    ap.add_argument('--disable-barline-mask', dest='disable_barline_mask', action='store_true', help='Disable barline overlap masking')"
        )
        changed = True
    # Argları parse ettikten sonra global ayarla
    if 'disable_barline_mask' in src and 'BARLINE_MASK_DISABLED = True' not in src:
        src = src.replace(
            'args = ap.parse_args()',
            "args = ap.parse_args()\n\n    # Apply disable barline mask\n    global BARLINE_MASK_DISABLED\n    if getattr(args, 'disable_barline_mask', False):\n        BARLINE_MASK_DISABLED = True"
        )
        changed = True
    # process_one içinde bar_mask seçimini güncelle
    pattern = "bar_mask = load_barline_mask_for_image(fname, '#00acc6')"
    replacement = "bar_mask = None if BARLINE_MASK_DISABLED else load_barline_mask_for_image(fname, '#00acc6')"
    if pattern in src:
        src = src.replace(pattern, replacement)
        changed = True
    if changed:
        with open(SCRIPT_FILE, 'w', encoding='utf-8') as f:
            f.write(src)
        print('Patch uygulandı: disable-barline-mask bayrağı eklendi.')
    else:
        print('Patch gerekli değil: değişiklikler zaten mevcut görünüyor.')

In [ ]:
# Hücre 10
# Patcher: Add --disable-barline-mask flag support in src/place_tmn_batch.py
# - Adds global toggle BARLINE_MASK_DISABLED
# - Adds argparse option --disable-barline-mask
# - Wires logic so barline mask can be bypassed safely
import os, re

candidates = [
    os.path.join('/content/omr_tmn', 'src', 'place_tmn_batch.py'),
    os.path.join('/content/omr_tmn', 'place_tmn_batch.py'),
    'src/place_tmn_batch.py'
]
script_path = next((p for p in candidates if os.path.isfile(p)), None)
print({'patch_target': script_path, 'exists': bool(script_path and os.path.isfile(script_path))})
if not script_path:
    raise FileNotFoundError('place_tmn_batch.py not found in expected locations')

with open(script_path, 'r', encoding='utf-8') as f:
    src = f.read()

changed = False

# 1) Insert global BARLINE_MASK_DISABLED after imports if missing
if 'BARLINE_MASK_DISABLED' not in src:
    lines = src.splitlines(True)
    last_import_idx = -1
    for i, ln in enumerate(lines):
        if ln.strip().startswith('import ') or ln.strip().startswith('from '):
            last_import_idx = i
    insert_line = "\nBARLINE_MASK_DISABLED = False\n"
    if last_import_idx >= 0:
        lines.insert(last_import_idx + 1, insert_line)
        src = ''.join(lines)
        changed = True

# 2) Ensure argparse flag exists
if '--disable-barline-mask' not in src:
    # Find parser initialization
    m = re.search(r"parser\s*=\s*argparse\.ArgumentParser\([^\)]*\)\s*\n", src)
    insert_point = m.end() if m else None
    if insert_point is None:
        # fallback: after first import argparse
        m2 = re.search(r"import\s+argparse\s*\n", src)
        insert_point = m2.end() if m2 else 0
    flag_line = "parser.add_argument('--disable-barline-mask', action='store_true', help='Disable barline masking during placement')\n"
    src = src[:insert_point] + flag_line + src[insert_point:]
    changed = True

# 3) Wire args to global toggle
if 'BARLINE_MASK_DISABLED = args.disable_barline_mask' not in src:
    m = re.search(r"args\s*=\s*parser\.parse_args\([^\)]*\)\s*\n", src)
    if m:
        ip = m.end()
        wire = "BARLINE_MASK_DISABLED = getattr(args, 'disable_barline_mask', False)\n"
        src = src[:ip] + wire + src[ip:]
        changed = True

# 4) Respect global in barline mask assignment
if 'BARLINE_MASK_DISABLED' in src and 'if BARLINE_MASK_DISABLED:' not in src:
    new_lines = []
    for ln in src.splitlines():
        new_lines.append(ln)
        if ln.strip().startswith('bar_mask') and '=' in ln:
            new_lines.append('if BARLINE_MASK_DISABLED:')
            new_lines.append('    bar_mask = None')
            changed = True
    src = '\n'.join(new_lines) + '\n'

if changed:
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(src)
    print('Patch uygulandı: --disable-barline-mask bayrağı ve mantığı eklendi')
else:
    print('Patch gerekmiyor: zaten uygulanmış görünüyor')
